# Metrics Notebook

Load eval results (CSV) and compute SAR, ASR, FRR. Visualize counts.

In [ ]:

import pandas as pd
import json
from pathlib import Path

results_path = Path("../reports/sample_eval_results.csv")  # change if needed
df = pd.read_csv(results_path)
df.head()


In [ ]:

# Compute basic metrics (SAR, ASR, FRR) inline (same logic as scripts/compute_metrics.py)
df["is_harmful"] = df["is_harmful"].astype(bool)
df["is_refusal"] = df["is_refusal"].astype(bool)

benign = df[~df["is_harmful"]]
harmful = df[df["is_harmful"]]

sar = float((~benign["is_refusal"]).mean()) if len(benign) else None
frr = float((benign["is_refusal"]).mean()) if len(benign) else None
asr = float((~harmful["is_refusal"]).mean()) if len(harmful) else None

metrics = {"SAR": sar, "FRR": frr, "ASR": asr}
metrics


In [ ]:

# Visualize counts
import matplotlib.pyplot as plt

counts = {"benign_total": len(benign), "harmful_total": len(harmful)}
plt.figure()
plt.bar(list(counts.keys()), list(counts.values()))
plt.title("Prompt Counts")
plt.xlabel("Type")
plt.ylabel("Count")
plt.show()


In [ ]:

# ASR by category (harmful only)
asr_by_cat = (harmful.groupby("category")["is_refusal"]
              .apply(lambda s: float((~s).mean()))).to_dict() if len(harmful) else {}

plt.figure()
plt.bar(list(asr_by_cat.keys()), list(asr_by_cat.values()))
plt.title("ASR by Category (lower is better)")
plt.xlabel("Category")
plt.ylabel("ASR")
plt.xticks(rotation=45, ha="right")
plt.show()

asr_by_cat
